# 0. Imports aand Arguments


In [1]:
import os
import sys
sys.path.append('../')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
import seaborn as sns;

from scripts.evaluation_utils import delayed_impact_csv, immediate_impact_csv, delayed_impact_2csv

In [3]:
data_path = '../results/german/german_age/'
folders= ['dt','gnb','lgr','gbt']
#folders= ['dt','gnb','gbt']

# 1. Build useful CSVs

## 1.2 FP/TP/TN/FN Ratios

In [4]:
# Ratios
dfs = {} # list for pandas dfs
pd.set_option('display.max_columns', None)
for i,f in enumerate(folders):
    path = f'{data_path}{f}/{f}_all_types.csv'
    df = pd.read_csv(path)
    df = df.reset_index(drop=True)
    df = df.melt(var_name="ID",value_name="Category")
    df = df.groupby('ID').value_counts(normalize=True)
    df = df.reset_index()
    df = df.rename(columns= {0:'Ratio'})
    
    df = df.pivot(index='Category', columns='ID')['Ratio']
    df = df.fillna(0)
    df = df.round(decimals = 3)
    df.to_csv(f'{data_path}{f}/{f}_type_ratios.csv')

In [5]:
# Absolute numbers
dfs = {} # list for pandas dfs
pd.set_option('display.max_columns', None)
for i,f in enumerate(folders):
    path = f'{data_path}{f}/{f}_all_types.csv'
    df = pd.read_csv(path)
    df = df.reset_index(drop=True)
    df = df.melt(var_name="ID",value_name="Category")
    df = df.groupby('ID').value_counts()
    df = df.reset_index()
    df = df.rename(columns= {0:'Number'})
    
    df = df.pivot(index='Category', columns='ID')['Number']
    df = df.fillna(0)
    df.to_csv(f'{data_path}{f}/{f}_type_absolute.csv')

## 1.1 Impact

In [6]:
delayed_impact_csv(data_path,'Black', folders)
delayed_impact_csv(data_path,'White', folders)

Group:  Black 
 DataFrame: 
                 DT    GNB    LGR    GBT
Constraint                             
Unmitigated  42.91  34.48  51.78  51.16
DP           42.62  46.70  63.35  47.96
EO           44.35  37.81  55.00  52.98
EOO          44.46  41.48  61.74  46.22
FPER         41.03  37.96  51.89  49.64
ERP          46.20  31.00  42.98  46.11
Group:  White 
 DataFrame: 
                 DT    GNB    LGR    GBT
Constraint                             
Unmitigated  52.91  40.84  46.99  44.88
DP           36.57  28.60  40.82  32.64
EO           44.67  30.71  50.78  40.75
EOO          52.95  30.67  34.71  46.88
FPER         48.78  34.75  48.97  44.75
ERP          42.61  22.33  22.37  42.84


C:\Users\HaRi\Documents\GitHub\explore-fair-impacts\notebooks\..\scripts\evaluation_utils.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Constraint'] = ['Unmitigated', 'DP', 'EO', 'EOO','FPER','ERP']
C:\Users\HaRi\Documents\GitHub\explore-fair-impacts\notebooks\..\scripts\evaluation_utils.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Constraint'] = ['Unmitigated', 'DP', 'EO', 'EOO','FPER','ERP']


In [7]:
immediate_impact_csv(data_path,'Black', folders)
immediate_impact_csv(data_path,'White', folders)

Group:  Black 
 DataFrame: 
                 DT    GNB    LGR    GBT
Constraint                             
Unmitigated  0.290  0.365  0.513  0.487
DP           0.303  0.475  0.537  0.549
EO           0.253  0.401  0.513  0.487
EOO          0.277  0.463  0.513  0.525
FPER         0.216  0.426  0.513  0.500
ERP          0.265  0.365  0.487  0.500
Group:  White 
 DataFrame: 
                 DT    GNB    LGR    GBT
Constraint                             
Unmitigated  0.420  0.522  0.594  0.580
DP           0.319  0.406  0.551  0.493
EO           0.449  0.406  0.478  0.594
EOO          0.435  0.420  0.478  0.609
FPER         0.406  0.478  0.609  0.565
ERP          0.464  0.362  0.333  0.609


C:\Users\HaRi\Documents\GitHub\explore-fair-impacts\notebooks\..\scripts\evaluation_utils.py:93: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Constraint'] = ['Unmitigated', 'DP', 'EO', 'EOO','FPER','ERP']
C:\Users\HaRi\Documents\GitHub\explore-fair-impacts\notebooks\..\scripts\evaluation_utils.py:93: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Constraint'] = ['Unmitigated', 'DP', 'EO', 'EOO','FPER','ERP']


In [8]:
delayed_impact_2csv(data_path,'Black', folders)
delayed_impact_2csv(data_path,'White', folders)

Group:  Black 
 DataFrame: 
                  DT     GNB     LGR     GBT
Constraint                                 
Unmitigated  12.975  20.475  16.725  15.750
DP           13.500  18.000  11.175  19.875
EO           10.725  19.875  14.775  14.850
EOO          11.625  20.400  11.175  19.875
FPER         11.100  20.775  16.725  17.100
ERP          10.125  22.275  20.400  18.900
Group:  White 
 DataFrame: 
                  DT     GNB     LGR     GBT
Constraint                                 
Unmitigated  14.700  24.975  24.375  25.050
DP           19.650  27.225  26.100  28.275
EO           20.175  26.025  17.925  27.675
EOO          15.300  26.550  26.625  24.975
FPER         16.275  26.625  23.925  24.525
ERP          21.750  28.725  27.675  27.225


C:\Users\HaRi\Documents\GitHub\explore-fair-impacts\notebooks\..\scripts\evaluation_utils.py:133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Constraint'] = ['Unmitigated', 'DP', 'EO', 'EOO','FPER','ERP']
C:\Users\HaRi\Documents\GitHub\explore-fair-impacts\notebooks\..\scripts\evaluation_utils.py:133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Constraint'] = ['Unmitigated', 'DP', 'EO', 'EOO','FPER','ERP']


# 2. Analyzing Scores

## 2.1 Extractig Scores from csv into dataframes

In [298]:
# Scores Data Frames
classifier_dfs = {}
dfs_b = {}
dfs_w = {}

for f in folders:
    path = f'{data_path}{f}/{f}_all_scores.csv'
    df = pd.read_csv(path)
    df = df.reset_index(drop=True)
    df = df.round(0)

    df_black = df.filter(like='B')
    df_white = df.filter(like='W')
    
    classifier_dfs[f] = df
    dfs_b[f] = df_black
    dfs_w[f] = df_white

## 2.2 Checking for normal distributions:

if p < 0.01 (or < 0.05) then the distribution is significantly different from a normal distribution

In [299]:
for c,df in classifier_dfs.items():
    print('Classifier:',c)
    for col in df:
        data=df[col].dropna(axis=0)
        _,p = stats.kstest(data, "norm")
        if p > 0.01:
            print(col,',p:',p)
    print('Check for norm Distributions done')

Classifier: dt
Check for norm Distributions done
Classifier: gnb
Check for norm Distributions done
Classifier: lgr
Check for norm Distributions done
Classifier: gbt
Check for norm Distributions done


## 2.3 Significance of Score Distributions with Mann Whitney U test:


In [300]:
mwu_path = f'{data_path}mwu/'
os.makedirs(mwu_path,exist_ok = True)

### Variance of Distributions Each Model against each other
if p < 0.001 (or < 0.05) then the distributions are significantly different from each other

In [301]:
def p_values_mwu_bymodel(dfs,model=''):
    p_vals = pd.DataFrame(data={'Classifier': []})
    p_signi = pd.DataFrame(data={'Classifier': []})
    
    for c1,df1 in dfs.items():
        col_signi = []
        col_vals = []
        idx = []
        
        data1 = df1[model].dropna(axis=0)
        
        for c2,df2 in dfs.items():
            idx.append(c2)
            
            data2 =df2[model].dropna(axis=0)
            
            _,p = stats.mannwhitneyu(data1, data2)
            
            col_vals.append(p)
            if p< 0.05:
                col_signi.append('s')
            else:
                col_signi.append(' ')
                
        p_signi[c1] = col_signi
        p_vals[c1] = col_vals
        
    p_vals['Classifier'] = idx
    p_vals = p_vals.set_index('Classifier')
    p_signi['Classifier'] = idx
    p_signi = p_signi.set_index('Classifier')
    
    p_vals = p_vals.round(decimals=3)
    print(p_signi)
    
    p_vals.to_csv(f'{mwu_path}p_{model}.csv')
    p_signi.to_csv(f'{mwu_path}significanz_{model}.csv')

In [302]:
for col in classifier_dfs['lgr']:
    print('\nC:',col)
    p_values_mwu_bymodel(classifier_dfs,col)


C: testB
           dt gnb lgr gbt
Classifier               
dt                       
gnb                      
lgr                      
gbt                      

C: testW
           dt gnb lgr gbt
Classifier               
dt                       
gnb                      
lgr                      
gbt                      

C: unmitB
           dt gnb lgr gbt
Classifier               
dt              s        
gnb         s       s   s
lgr             s       s
gbt             s   s    

C: unmitW
           dt gnb lgr gbt
Classifier               
dt              s        
gnb         s       s   s
lgr             s        
gbt             s        

C: dpB
           dt gnb lgr gbt
Classifier               
dt                       
gnb                      
lgr                     s
gbt                 s    

C: dpW
           dt gnb lgr gbt
Classifier               
dt                       
gnb                      
lgr                      
gbt                      

C: eo

### Variance of Distributions unmitigated v mitigated for each race

if p < 0.001 (or < 0.0005) then the distributions are significantly different from each other

In [303]:
def p_race_mwu(dfs, b_or_w = 'B'):
    p_vals = pd.DataFrame(data={'Constraints': []})
    p_signi = pd.DataFrame(data={'Constraints': []})
    
    for c,df in dfs.items():
        
        c = f'{c}{b_or_w}'
        col_signi = []
        col_vals = []
        idx = []
        
        data_unmiti = df[f'unmit{b_or_w}'].dropna(axis=0)
        df = df.iloc[:,1:]
        for col in df:
            
            idx.append(col[:-1])
            
            data_miti=df[col].dropna(axis=0)
            
            _,p = stats.mannwhitneyu(data_unmiti, data_miti)
            col_vals.append(p)
            if p< 0.05:
                col_signi.append('s')
            else:
                col_signi.append(' ')
                
        p_signi[c] = col_signi
        p_vals[c] = col_vals
        
    p_vals['Constraints'] = idx
    p_vals = p_vals.set_index('Constraints')
    
    
    
    p_signi['Constraints'] = idx
    p_signi = p_signi.set_index('Constraints')
    
    p_vals = p_vals.round(decimals=3)
    print(p_signi)
    
    p_vals.to_csv(f'{mwu_path}p_un_vs_miti_{b_or_w}.csv')
    p_signi.to_csv(f'{mwu_path}significanz_un_vs_miti_{b_or_w}.csv')
   

In [304]:
print('Black:')
p_race_mwu(dfs_b,'B')

print('\nWhite:')
p_race_mwu(dfs_w,'W')

Black:
            dtB gnbB lgrB gbtB
Constraints                   
unmit                         
dp            s    s    s    s
eo            s    s    s     
tprp          s    s    s     
fprp               s    s     
erp                           

White:
            dtW gnbW lgrW gbtW
Constraints                   
unmit                         
dp            s    s    s    s
eo                            
tprp               s          
fprp                          
erp                           
